In [1]:
import geopandas as gpd
import geodatasets
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from shapely.geometry import Polygon, LineString, MultiPoint, Point
import psycopg2
import csv
import pandas as pd
# import pymongo
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
client = MongoClient()

import json
import dateutil
from dateutil import parser
import glob
import os
from os.path import isfile, join
path = os.getcwd() 
data_path = path

In [2]:
def copy_to_postgres(filename, pg_cursor, table_name):
    with open(filename, 'r', encoding='utf-8-sig') as file:
        next(file)
        pg_cursor.copy_from(file, table_name, sep=',') 
        
def copy_expert_to_postgres(filename, pg_cursor, table_name):
    with open(filename, 'r', encoding='utf-8-sig') as file:
        next(file)
        pg_cursor.copy_expert('''COPY {table_name} FROM STDIN WITH CSV HEADER''', file)

In [3]:
########################### MUST CREATE CONNECTION to Postgres before any calls are made
pg_conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="darkdreamer17?",  # change this to your own password
        host="localhost",
        port="5432"
    )
pg_conn.autocommit = True
pg_cursor = pg_conn.cursor()


# create test scores table
create_testscores_table = '''
    CREATE TABLE IF NOT EXISTS ritscores (
    SchoolID INT,
    SchoolName VARCHAR(255),
    Network VARCHAR(255),
    Subject VARCHAR(255),
    Grade VARCHAR(50),
    NumStudents INT,
    AvgRITscore NUMERIC(5, 1),
    AtOrAboveNationalAvg NUMERIC(4, 1),
    NationalAttainmentPercentile INT
    );
    '''
pg_cursor.execute(create_testscores_table)


# create redlined schools table
# create_schooldistrict_table = '''
#     CREATE TABLE schoolsdata (
#     School_ID INT PRIMARY KEY,
#     Short_Name VARCHAR(255),
#     holc_id VARCHAR(5),
#     holc_grade VARCHAR(1),
#     Grade_Cat VARCHAR(2)
#     );
#     '''
# pg_cursor.execute(create_schooldistrict_table)


# import csv data to tables
with open('chicago_testscores_2019.csv', 'r', encoding='utf-8-sig') as file:
        next(file)
        pg_cursor.copy_expert('''COPY ritscores FROM STDIN WITH CSV HEADER''', file)
# copy_to_postgres('../data/Schools.csv', pg_cursor, 'schoolsdata')


####################### Querying #######################
# first, select: school short name, 


####################### Close the PostgreSQL cursor and connection - MUST BE RUN AFTER ANY CALLS TO POSTGRES
if pg_cursor is not None:
    pg_cursor.close()
if pg_conn is not None:
    pg_conn.close()

In [4]:
# use geopandas to map school locations onto modern neighborhoods
# import Schools.csv as geopandas dataframe